In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

Леонов, датамайнинг

### 1. Описание задачи

Ссылка на контест: https://www.kaggle.com/c/petfinder-pawpularity-score

**Суть задачи**

Есть сайт для поиска домашних животных, на котором размещены объявления с фотками и некоторой метаинфой о питомцах. На базе активности пользователей был разработан индекс популяроности данного животного **Pawpularity** $\in [0 \text{ (bad)},100 \text{ (good)}]$. 

**Дано (features):** фотка и табличные данные про животное.

**Цель (target):** предсказать **Pawpularity**.

Начнем с того, что загрузим и почистим данные.

In [ ]:
table_path = '../input/petfinder-pawpularity-score/train.csv'
table_path_test = '../input/petfinder-pawpularity-score/test.csv'
photo_path = '../input/petfinder-pawpularity-score/train'

##### Про целевую переменную (Pawpularity)


    The Pawpularity Score is derived from each pet profile's page view statistics at the listing pages, using an algorithm that normalizes the traffic data across different pages, platforms (web & mobile) and various metrics.
    Duplicate clicks, crawler bot accesses and sponsored profiles are excluded from the analysis.
    
##### Про features (они все бинарные)


    Focus - Pet stands out against uncluttered background, not too close / far.
    
    Eyes - Both eyes are facing front or near-front, with at least 1 eye / pupil decently clear.
    
    Face - Decently clear face, facing front or near-front.
    
    Near - Single pet taking up significant portion of photo (roughly over 50% of photo width or height).
    
    Action - Pet in the middle of an action (e.g., jumping).
    
    Accessory - Accompanying physical or digital accessory / prop (i.e. toy, digital sticker), excluding collar and leash.
    
    Group - More than 1 pet in the photo.
    
    Collage - Digitally-retouched photo (i.e. with digital photo frame, combination of multiple photos).
    
    Human - Human in the photo.
    
    Occlusion - Specific undesirable objects blocking part of the pet (i.e. human, cage or fence). Note that not all blocking objects are considered occlusion.
    
    Info - Custom-added text or labels (i.e. pet name, description).
    
    Blur - Noticeably out of focus or noisy, especially for the pet’s eyes and face. For Blur entries, “Eyes” column is always set to 0.


Сразу отметим, что данных для такой задачи очень мало $(9912)$, также регрессия по фоткам всегда считалась нетривиальной задачей. По-хорошему, следует найти схожую задачу, взять предобученную сеть и уже ее применить (transfer learning), но к сожалению, на данный момент еще не хватает навыков реализовать этот сценарий, поэтому попробуем некоторые простые подходы и посмотрим, что выйдет.

In [ ]:
df = pd.read_csv(table_path)
df.shape # 12 фичей (не считая Id и Pawpularity)

In [ ]:
# удалим экстремальные значения

q1 = df.Pawpularity.quantile(0.01)
q2 = df.Pawpularity.quantile(0.99)

df = df[~((df.Pawpularity <= q1) |(df.Pawpularity >= q2))]

In [ ]:
df.shape

In [ ]:
df.isna().any().sum() # данные без пропусков

Теперь про фотки

In [ ]:
# считаем фотки

import cv2
import os
from skimage import io
from skimage.transform import resize

X_photo = [] # в таком же порядке
# расположены будут, как и в df расположена инфа о фотке

for img_id in df.Id:
    
    img = io.imread(photo_path+'/'+ str(img_id)+'.jpg',
                    as_gray=True)# и превращаем в черное-белые  
    
    # пока будем просто ресайзить к одному размеру (28,28)
    X_photo.append(cv2.resize(img,(28,28)))   

In [ ]:
plt.imshow(X_photo[0]) # пример фотки

In [ ]:
np.max(X_photo),np.min(X_photo) # черное-белые уже отнормированы

### 2. Регрессия на числовых данных (в смысле, не фотки)

Сначала просто попробуем сделать Ridge-регрессию (делаем регуляризацию, потому что много коррелирующих признаков)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.heatmap(df.corr(),annot=True,
           annot_kws={"fontsize":17},cmap="PuBu")

$$\min_{w} || X w - y||_2^2 + \alpha ||w||_2^2, \; \alpha \geq 0 $$


Чтобы подобрать $\alpha$ используем кросс-проверку, то есть разбиваем выборку на $k$ подмножества, каждое из которых будем использовать для проверки, а оставшиеся - для обучения. Получаем $k$ моделей, для которых можно посчитать среднюю ошибку - получаем некоторую объекивную оценку того, как модель работает при заданном $\alpha$.

Leave-one-out кросс-проверка (LOOCV) - частный крайний случай, когда мы строим модель на всех данных, кроме одного элемента выборки. Именно он по-умолчанию стоит в RidgeCV.


In [ ]:
Xnames = [x for x in df.columns if x not in ('Id','Pawpularity')]
ynames = ['Pawpularity']

X = df[Xnames]
y = df[ynames]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=404)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
model = RidgeCV(alphas=np.linspace(0.001,1000,1000))
model.fit(X_train,np.array(y_train).ravel())
model.alpha_

In [ ]:
y_pred = model.predict(X_test)
r2_score(y_test,y_pred), mean_squared_error(y_test,y_pred)

Получилось модель, чуть лучше чем константа (среднее)

In [ ]:
y_pred_mean = np.ones(len(y_test)) * np.mean(y_test)[0]

r2_score(y_test,y_pred_mean), mean_squared_error(y_test,y_pred_mean)

### 3. Регрессия на фотках (сверточная нейронная сеть)

In [ ]:
import tensorflow as tf
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
X_photo = np.array(X_photo)
y = df[ynames]
X_photo = X_photo.reshape(-1, 28, 28, 1) # нужно для keras

X_train, X_test, y_train, y_test = train_test_split(X_photo,y,
                                                    test_size=0.2,
                                                    random_state=404)

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3),
                                 activation="relu",
                                 input_shape=(28, 28, 1)))

model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), 
                                 activation="relu"))

model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Conv2D(64, (3, 3), 
                                 activation="relu"))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, 
                                activation="relu"))

model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
model.compile(loss='mse',
             optimizer='adam')
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train,
         y_train,
         batch_size=64,
         epochs=30,callbacks=[callback],
         validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred)

MSE сверточной нейронной сети хуже, чем просто RidgeCV и даже хуже, чем константный прогноз (средним).

### 4. Классификация на числовых данных (10 классов)

Попробуем "дискретизировать" задачу: разобъем целевую переменную на группы.

In [ ]:
def classify(x):
    for i in range(10):
        if x >= 0 + 10*i and x < 10 + 10*i:
            return i
    return 9 # если там есть 100

In [ ]:
X = df[Xnames]
y = [classify(x) for x in df[ynames].values]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)

Для классификации используем ансамбль деревьев: выборка разбивается на подвыборки, на каждой из которых обучается дерево, а потом результат усредняется.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

Только четверть ответов угадывается правильно. Попробуем другие разбиения и найдет такое, при котором accuracy максимальное.

### 5. Бинарная классификация

In [ ]:
classify = lambda x: 0 if x <= 50 else 1

In [ ]:
X = df[Xnames]
y = [classify(x) for x in df[ynames].values]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test,y_pred)

Теперь построим еще две модели: отедельно для двух классов.

In [ ]:
# классификацию для [0;50]
classify = lambda x: 0 if x <= 25 else 1

X0 = df[df.Pawpularity <= 50][Xnames]
y0 = [classify(x) for x in df[df.Pawpularity <= 50][ynames].values]

X_train, X_test, y_train, y_test = train_test_split(X0,y0,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)

In [ ]:
model0 = RandomForestClassifier()
model0.fit(X_train,y_train)
y_pred = model0.predict(X_test)
accuracy_score(y_test,y_pred)

- Регрессия 0

In [ ]:
# теперь регрессию для [0;25]

# сильно урежем данные
df1 = df[df.Pawpularity <= 25]
q1 = df1.Pawpularity.quantile(0.3)
q2 = df1.Pawpularity.quantile(0.7)
df1 = df1[~((df1.Pawpularity <= q1) |(df1.Pawpularity >= q2))]

X0 = df1[Xnames]
y0 = df1[ynames]

X_train, X_test, y_train, y_test = train_test_split(X0,y0,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)



In [ ]:
reg0 = RidgeCV(alphas=np.linspace(0.001,1000,1000))
reg0.fit(X_train,np.array(y_train).ravel())
reg0.alpha_

In [ ]:
y_pred = reg0.predict(X_test)
r2_score(y_test,y_pred), mean_squared_error(y_test,y_pred)

- Регрессия 1

In [ ]:
# теперь регрессию для (25;50]

# сильно урежем данные
df1 = df[((df.Pawpularity > 25) & (df.Pawpularity <= 50))]
q1 = df1.Pawpularity.quantile(0.15)
q2 = df1.Pawpularity.quantile(0.85)
df1 = df1[~((df1.Pawpularity <= q1) |(df1.Pawpularity >= q2))]

X0 = df1[Xnames]
y0 = df1[ynames]

X_train, X_test, y_train, y_test = train_test_split(X0,y0,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)



In [ ]:
reg1 = RidgeCV(alphas=np.linspace(0.001,1000,1000))
reg1.fit(X_train,np.array(y_train).ravel())
reg1.alpha_

In [ ]:
y_pred = reg1.predict(X_test)
r2_score(y_test,y_pred), mean_squared_error(y_test,y_pred)

----

In [ ]:
# классификацию для (50;100]
classify = lambda x: 0 if x <= 75 else 1

X0 = df[df.Pawpularity > 50][Xnames]
y0 = [classify(x) for x in df[df.Pawpularity > 50][ynames].values]

X_train, X_test, y_train, y_test = train_test_split(X0,y0,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)

In [ ]:
model1 = RandomForestClassifier()
model1.fit(X_train,y_train)
y_pred = model1.predict(X_test)
accuracy_score(y_test,y_pred)

- Регрессия 2

In [ ]:
# теперь регрессию для (50;75]

# сильно урежем данные
df1 = df[((df.Pawpularity > 50) & (df.Pawpularity <= 75))]
q1 = df1.Pawpularity.quantile(0.2)
q2 = df1.Pawpularity.quantile(0.8)
df1 = df1[~((df1.Pawpularity <= q1) |(df1.Pawpularity >= q2))]

X0 = df1[Xnames]
y0 = df1[ynames]

X_train, X_test, y_train, y_test = train_test_split(X0,y0,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)



In [ ]:
reg2 = RidgeCV(alphas=np.linspace(0.001,1000,1000))
reg2.fit(X_train,np.array(y_train).ravel())
reg2.alpha_

In [ ]:
y_pred = reg2.predict(X_test)
r2_score(y_test,y_pred), mean_squared_error(y_test,y_pred)

- Регрессия 3

In [ ]:
# теперь регрессию для (75;100]

# сильно урежем данные
df1 = df[((df.Pawpularity > 75) & (df.Pawpularity <= 100))]
q1 = df1.Pawpularity.quantile(0.01)
q2 = df1.Pawpularity.quantile(0.99)
df1 = df1[~((df1.Pawpularity <= q1) |(df1.Pawpularity >= q2))]

X0 = df1[Xnames]
y0 = df1[ynames]

X_train, X_test, y_train, y_test = train_test_split(X0,y0,
                                                    test_size=0.2,
                                                    random_state=404)
np.shape(X_train),np.shape(y_train),np.shape(X_test),np.shape(y_test)



In [ ]:
reg3 = RidgeCV(alphas=np.linspace(0.001,1000,1000))
reg3.fit(X_train,np.array(y_train).ravel())
reg3.alpha_

In [ ]:
y_pred = reg3.predict(X_test)
r2_score(y_test,y_pred), mean_squared_error(y_test,y_pred)

В конце концов:

Сначала два раза применяем классификацию и в зависимости от того, что у нас получилось, применяем одну из четырех регрессий.


In [ ]:
def fin_model(x):
    
    if model.predict(x) == 0:
        if model0.predict(x) == 0:
            return reg0.predict(x)
        else:
            return reg1.predict(x)
            
    else:
        if model1.predict(x) == 0:
            return reg2.predict(x)
        else:
            return reg3.predict(x)
        

In [ ]:
Xnames = [x for x in df.columns if x not in ('Id','Pawpularity')]
ynames = ['Pawpularity']

X = df[Xnames]
y = df[ynames]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=404)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
fin_model(X_test.iloc[[0]])

In [ ]:
y_pred = [fin_model(X_test.iloc[[i]]) for i in range(len(X_test))]


In [ ]:
mean_squared_error(y_test,y_pred)

### 6. Вывод

Из всех представленных подходов самым лучшим с точки зрения MSE ялвяется RidgeCV, ее и будем считать итоговой моделью.

In [ ]:
Xnames = [x for x in df.columns if x not in ('Id','Pawpularity')]
ynames = ['Pawpularity']

X = df[Xnames]
y = df[ynames]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=404)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
model = RidgeCV(alphas=np.linspace(0.001,1000,1000))
model.fit(X_train,np.array(y_train).ravel())
model.alpha_

In [ ]:
y_pred = model.predict(X_test)
r2_score(y_test,y_pred), mean_squared_error(y_test,y_pred)

---

In [ ]:
# вывод для kaggle
True_test_data = pd.read_csv(table_path_test)
answer = pd.DataFrame(model.predict(True_test_data.drop(['Id'],axis=1)),columns=['Pawpularity'])
pd.concat([True_test_data.Id,answer],axis=1).to_csv("submission.csv", index=False)